# MODEL TRAINING - testing the full approach - COGNITION
saving models in a different folder using pickle
**DO NOT RUN AGAIN**

In [49]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import uuid
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"/COGNITION"
dreamspath=datapath+"/DREAMS"
masspath=datapath+"/MASS"


## experiment id
we'll use it to create a folder at the end of the script

In [2]:
experimentId=str(uuid.uuid4())

## define a fixed samplerate and input dimension for NN

In [16]:
samplerate=200
inputDimension=601

## load cognition

In [4]:
signals,annotations, signalsMetadata = loadCOGNITIONSpindles(cognipath, returnSignals=True)

In [5]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
len(annotations)

2785

In [6]:
annotations.head()

,subjectId,channel,startTime,stopTime,duration,phase,labelerId,type,samplerate,startInd,stopInd
0,0001,C3,1938.132297,1939.202326,1.070029,2,0001,spindle,200,387626,387840
1,0001,C3,2047.217898,2048.112834,0.894936,2,0001,spindle,200,409444,409623
2,0001,C3,2170.856032,2171.634237,0.778205,3,0001,spindle,200,434171,434327
3,0001,C3,2219.766538,2220.894943,1.128405,3,0001,spindle,200,443953,444179
4,0001,C3,2263.482492,2264.357974,0.875482,3,0001,spindle,200,452696,452872


In [7]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,isOriginalSamplerate,database
0,COG001_canal_C3.txt,C3-M2,0001,39600,200,False,COGNITION
1,COG002_canal_C3.txt,C3-M2,0002,39600,200,False,COGNITION
2,COG003_canal_C3.txt,C3-M2,0003,39600,200,False,COGNITION
3,COG004_canal_C3.txt,C3-M2,0004,39600,200,False,COGNITION
4,COG005_canal_C3.txt,C3-M2,0005,39600,200,False,COGNITION


## Preprocess

broadband in 0-40Hz, previous experiments (up tp 50Hz, are now in folders as .../features_old/...)

In [8]:
def preprocessVector(vector,samplerate):
    #1. Lowpass 40Hz
    vector=filterBand(vector,[0,40],samplerate,filterOrder=4)
    #2. Z-score
    m=np.mean(vector)
    s=np.std(vector)
    vector=(vector-m)/s
    return vector

In [9]:
for subject,signal in signals.items():
    signals[subject]=preprocessVector(signal,samplerate)

## prepare slides and labels for training

In [11]:
signalsMetadata['excerptDimension']=signalsMetadata.apply(lambda row: int(row.duration*row.samplerate),axis=1)
allExtended=pd.DataFrame()
for ind,row in signalsMetadata.iterrows():
    thisExtended=pd.DataFrame({
        'trainIndex':np.arange(row.excerptDimension),
        'subjectId':row.subjectId
    })
    allExtended=pd.concat([allExtended,thisExtended])

In [12]:
allExtended

,trainIndex,subjectId
0,0,0001
1,1,0001
2,2,0001
3,3,0001
4,4,0001
...,...,...
7919995,7919995,0009
7919996,7919996,0009
7919997,7919997,0009
7919998,7919998,0009


In [13]:
usedAnnotations=annotations[annotations.labelerId=='0001'].reset_index(drop=True)

In [14]:
labels={}
for index, row in signalsMetadata.iterrows():
    labels[row.subjectId]=excerptAnnotationsToLabels(usedAnnotations[usedAnnotations.subjectId==row.subjectId],excerptDimension=row.excerptDimension)

In [15]:
#check
for index, row in signalsMetadata.iterrows():
    print(row.excerptDimension==len(labels[row.subjectId]))

True
True
True
True
True
True
True
True
True


In [17]:
slides={}
for index, row in signalsMetadata.iterrows():
    paddedSignal=padVectorBothSides(signals[row.subjectId],window2half(inputDimension),method='closest')
    slides[row.subjectId]=np_tricks.sliding_window_view(paddedSignal,(inputDimension,))

In [18]:
#check
for index, row in signalsMetadata.iterrows():
    print(row.excerptDimension==len(slides[row.subjectId]))

True
True
True
True
True
True
True
True
True


## load data split
LOOCV with 7 subjects

In [19]:
#by now we used fixed CV data splits instead
dataSplits=loadPickle("dataSplits_LOOCV.pkl")
dataSplits

,train,val,test
0,"[0004, 0005, 0006, 0007]","[0003, 0008]",0002
1,"[0002, 0005, 0006, 0008]","[0007, 0004]",0003
2,"[0002, 0003, 0005, 0007]","[0006, 0008]",0004
3,"[0002, 0003, 0007, 0008]","[0006, 0004]",0005
4,"[0002, 0003, 0007, 0008]","[0004, 0005]",0006
5,"[0002, 0004, 0005, 0008]","[0003, 0006]",0007
6,"[0002, 0005, 0006, 0007]","[0003, 0004]",0008


## define annotation criteria

In [20]:
np.unique(annotations.labelerId)

array(['0001'], dtype=object)

In [21]:
annotationCriteria=pd.DataFrame({
    'criteriumId':['0001'],
    'criteriumName':['E1'],
    'labelerIdList':[['0001']]
})

In [22]:
annotationCriteria

,criteriumId,criteriumName,labelerIdList
0,0001,E1,[0001]


In [23]:
experimentModels=pd.merge(annotationCriteria,dataSplits,how='cross')

In [24]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test
0,0001,E1,[0001],"[0004, 0005, 0006, 0007]","[0003, 0008]",0002
1,0001,E1,[0001],"[0002, 0005, 0006, 0008]","[0007, 0004]",0003
2,0001,E1,[0001],"[0002, 0003, 0005, 0007]","[0006, 0008]",0004
3,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0006, 0004]",0005
4,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0004, 0005]",0006
5,0001,E1,[0001],"[0002, 0004, 0005, 0008]","[0003, 0006]",0007
6,0001,E1,[0001],"[0002, 0005, 0006, 0007]","[0003, 0004]",0008


## model fitting

prepare the NN

In [27]:
X_input = layers.Input((601))
X = layers.Dense(200,activation='relu')(X_input)
X = layers.Dense(100,activation='relu')(X)
X = layers.Dense(50,activation='relu')(X)
X = layers.Dense(25,activation='relu')(X)
X = layers.Dense(5,activation='relu')(X)
X = layers.Dense(1,activation='sigmoid')(X)
model = models.Model(inputs=X_input,outputs=X)

In [28]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 601)]             0         
                                                                 
 dense (Dense)               (None, 200)               120400    
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dense_3 (Dense)             (None, 25)                1275      
                                                                 
 dense_4 (Dense)             (None, 5)                 130       
                                                                 
 dense_5 (Dense)             (None, 1)                 6     

In [29]:
row=experimentModels.iloc[0]

In [30]:
row

criteriumId                          0001
criteriumName                          E1
labelerIdList                      [0001]
train            [0004, 0005, 0006, 0007]
val                          [0003, 0008]
test                                 0002
Name: 0, dtype: object

In [31]:
allExtended.columns

Index(['trainIndex', 'subjectId'], dtype='object')

In [39]:
trainExtended=allExtended[allExtended.subjectId.isin(row.train)].reset_index(drop=True)
trainGenerator=spinGen(slides,labels,trainExtended)

In [40]:
valExtended=allExtended[allExtended.subjectId.isin(row.val)].reset_index(drop=True)
valGenerator=spinGen(slides,labels,valExtended)

In [43]:
optimizer = optimizers.Adam()
loss_fn = tf.keras.losses.binary_crossentropy
model.compile(optimizer, loss_fn)
history = model.fit(trainGenerator,validation_data=valGenerator,epochs=1)

30937/30937 [==============================] - 226s 7ms/step - loss: 0.0140 - val_loss: 0.0145


In [44]:
history.history

{'loss': [0.014004088938236237], 'val_loss': [0.014458983205258846]}

In [50]:
DLmodels=[]
modelIds=[]
for index, row in experimentModels.iterrows():
    print('**********************')
    print(str(index+1)+' of '+str(len(experimentModels)))
    #model definition
    X_input = layers.Input((inputDimension))
    X = layers.Dense(200,activation='relu')(X_input)
    X = layers.Dense(100,activation='relu')(X)
    X = layers.Dense(50,activation='relu')(X)
    X = layers.Dense(25,activation='relu')(X)
    X = layers.Dense(5,activation='relu')(X)
    X = layers.Dense(1,activation='sigmoid')(X)
    model = models.Model(inputs=X_input,outputs=X)

    #train generator
    trainExtended=allExtended[allExtended.subjectId.isin(row.train)].reset_index(drop=True)
    trainGenerator=spinGen(slides,labels,trainExtended)
    #val generator
    valExtended=allExtended[allExtended.subjectId.isin(row.val)].reset_index(drop=True)
    valGenerator=spinGen(slides,labels,valExtended)
    #model fit
    optimizer = optimizers.Adam()
    loss_fn = tf.keras.losses.binary_crossentropy
    model.compile(optimizer, loss_fn)
    history = model.fit(trainGenerator,validation_data=valGenerator,epochs=1)

    DLmodels.append(model)
    modelIds.append(str(uuid.uuid4()))


**********************
1 of 7
30937/30937 [==============================] - 227s 7ms/step - loss: 0.0148 - val_loss: 0.0143
**********************
2 of 7
30937/30937 [==============================] - 227s 7ms/step - loss: 0.0102 - val_loss: 0.0191
**********************
3 of 7
30937/30937 [==============================] - 228s 7ms/step - loss: 0.0180 - val_loss: 0.0057
**********************
4 of 7
30937/30937 [==============================] - 227s 7ms/step - loss: 0.0153 - val_loss: 0.0143
**********************
5 of 7
30937/30937 [==============================] - 226s 7ms/step - loss: 0.0151 - val_loss: 0.0161
**********************
6 of 7
30937/30937 [==============================] - 227s 7ms/step - loss: 0.0123 - val_loss: 0.0182
**********************
7 of 7
30937/30937 [==============================] - 220s 7ms/step - loss: 0.0136 - val_loss: 0.0262


In [51]:
experimentModels["modelId"]=modelIds

In [52]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId
0,0001,E1,[0001],"[0004, 0005, 0006, 0007]","[0003, 0008]",0002,04a1db9a-a55d-4c61-9aca-e9176cf39cfe
1,0001,E1,[0001],"[0002, 0005, 0006, 0008]","[0007, 0004]",0003,7660a83c-462e-480a-9c21-b9a672763ef1
2,0001,E1,[0001],"[0002, 0003, 0005, 0007]","[0006, 0008]",0004,08c6cc01-0d83-44b4-a587-9ca9a36dfb14
3,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0006, 0004]",0005,721fa0ce-6946-45a2-8332-b310eefd8481
4,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0004, 0005]",0006,c45b44b5-f5bb-4ccf-bda7-e373c18b55ef
5,0001,E1,[0001],"[0002, 0004, 0005, 0008]","[0003, 0006]",0007,1aa18dd2-aaf7-4ed0-a335-18698a433632
6,0001,E1,[0001],"[0002, 0005, 0006, 0007]","[0003, 0004]",0008,a5692535-9a66-4d75-836a-8effb7c1423c


## save results

In [57]:
#create parent folder
experimentpath=datapath+"/experiments/"+experimentId
os.mkdir(experimentpath)
#save each of the models
for name, model in zip(modelIds,DLmodels):
    dumpPickle(experimentpath+"/"+name+".pkl",model)
#save experiment information
dumpPickle(experimentpath+"/experimentModels.pkl",experimentModels)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dense_3
......vars
.........0
.........1
...layers\dense_4
......vars
.........0
.........1
...layers\dense_5
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
.........21
.........22
.........23
.........24
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-09-01 10:25:59         4112
metadata.json                         

In [58]:
#you need to use this in the following parts of the experiment
print(experimentId)

ca058290-1b4e-4883-94b8-537361a7d7e8
